# Preface

Astro Teller, Google X, described failure as something to be celebrated, as it is a necessary step towards success. Per Teller:
   
>But that’s the point: you can’t get to the good ideas without spending a lot of time warming up your creativity with a bunch of bad ones. Most people never develop those muscles because they’re too worried about what someone else is going to think of them. It’s a shame, because a terrible idea is often the cousin of a good idea, and a great one is the neighbor of that.

In fact, Google not only accepts failure, but *rewards* its employees for it:

>At X, when a team kills their own project because they find a fatal flaw, they often get a bonus, Teller says–not one so big as to encourage people to kill projects without good reason, but not pocket change either. Even more importantly, they are applauded by peers and supervisors at all-team meetings and often rewarded with a few months of exploration time to work out what their next project will be.
   
With this being said, I would like to begin this analysis by proudly declaring:

### I have failed. 

Source: https://www.wired.com/2016/04/the-head-of-x-explains-how-to-make-audacity-the-path-of-least-resistance/#.pgciatmtq

# The Question

There's a clear and obvious trend in all industries towards utilizing large swathes of data to glean information that was previously seen as too demanding and labor-intensive. It's also clear that some industries have adapted these practices quicker than others. 

Hollywood is typically not seen as being on the forefront of technology and/or new practices - it took _ years for the industry to adopt _ mm film, and even now, some production companies still utilize film cameras (not as an artistic choice). 

The entertainment industry as a whole has a lot to gain from modern machine learning efforts, though, due to the sheer amount of (relatively) unused data that they're sitting on. Given the amount of information, it should be reasonable to be able to use this historical data as the basis to make predictions on the future. In particular, I want to explore if there are certain identifiable traits or trends in network television shows that contribute to it's success (or lack thereof). 

In [1]:
import pandas as pd
import re
import wikipedia
import requests
from bs4 import BeautifulSoup
import time
import numpy as np

In [2]:
# https://medium.com/@Alexander_H/scraping-wikipedia-with-python-8000fc9c9e6c

# Where to gather data?

First things first - I need to find out how to get data to explore this question. I would want as much information about the shows as possible, including individual episodes if available. Another large factor in my initial search was the inclusion of viewership numbers (i.e. how many people watched each episode). After some research, it appears that this information is really only available in a database capacity from Nielsen. The information is reported weekly by TVByTheNumbers, but was removed from their API by Zap2it as requested by Nielsen. 

Because tv is such a profit-driven industry, I couldn't find any free databases to query the information I wanted directly. Even when looking for only show information, IMDB, which is the most intuitive possible source, does not have a public API.

Despite this apparent lack of reliable information, I was aware that Wikipedia has (un?)surprisingly extensive information about television shows. I settled on trying to scrape tv shows on Wikipedia by network, then grabbing a list of show names from the network pages. 

# Function to grab all links found in tables on specified wikipedia pages

I can create a function to use BeautifulSoup to search on webpages for lists, then grab the show names from those lists. In that same function, I will also make the show name back into a wikipedia link with the assumption that I will need to access the pages to grab the tables.

Although this was a quick and easy solution, it does have the downside of grabbing all the unrelated things that are in lists, as well. I will have to filter these out later using RegEx.

In [4]:
def get_network_links(url):
    html = requests.get(str(url))
    b = BeautifulSoup(html.text, 'lxml')
    links = []

    for i in b.find_all(name = 'li'):
        for link in i.find_all('a', href=True):
            links.append(link['href'])
    links = links[57:]
    network_links = ['https://en.wikipedia.org' + i for i in links]

    return network_links
    # Need to remove cites, external links, related links, List links

In [8]:
abc_links = get_network_links('https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_American_Broadcasting_Company')
fox_links = get_network_links('https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_Fox')
nbc_links = get_network_links('https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_NBC')
cbs_links = get_network_links('https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_CBS')
hbo_links = get_network_links('https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_HBO')
amc_links = get_network_links('https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_AMC')
cw_links = get_network_links('https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_The_CW')
as_links = get_network_links('https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_Adult_Swim')
cc_links = get_network_links('https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_Comedy_Central')
disney_links = get_network_links('https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_Disney_Channel')
syfy_links = get_network_links('https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_Syfy')

I could very feasibly make another function that iterates through a list of network links, then calls this function. 

I want to take this list of television show links to then grab the individual episode information from. I very quickly ran into an issue with this, though - due to the openly editable nature of Wikipedia, the formatting is very inconsistent. For example, certain Wiki pages have the information under the [_tv show link_] url, while others use [_List of tv show episodes_] format. Others still use [_tv show Season(#)_] format. Even within these urls, there are differences in what is contained in the tables themselves. All-in-all, the information is too inconsistent and requires too much munging, so I'll have to find a way to get show information another way. I do have a useable list of show names now, though. 

In [6]:
import pickle

In [9]:
with open("./Assets_&_Data/fox_links.pickle","wb") as fp:
    pickle.dump(fox_links, fp)
    
with open("./Assets_&_Data/cbs_links.pickle","wb") as fp:
    pickle.dump(cbs_links, fp)
    
with open("./Assets_&_Data/nbc_links.pickle","wb") as fp:
    pickle.dump(nbc_links, fp)
    
with open("./Assets_&_Data/hbo_links.pickle","wb") as fp:
    pickle.dump(hbo_links, fp)
    
with open("./Assets_&_Data/amc_links.pickle","wb") as fp:
    pickle.dump(amc_links, fp)
    
with open("./Assets_&_Data/cw_links.pickle","wb") as fp:
    pickle.dump(cw_links, fp)

with open("./Assets_&_Data/as_links.pickle","wb") as fp:
    pickle.dump(as_links, fp)

with open("./Assets_&_Data/cc_links.pickle","wb") as fp:
    pickle.dump(cc_links, fp)

with open("./Assets_&_Data/disney_links.pickle","wb") as fp:
    pickle.dump(disney_links, fp)
    
with open("./Assets_&_Data/syfy_links.pickle","wb") as fp:
    pickle.dump(syfy_links, fp)

with open("./Assets_&_Data/abc_links.pickle","wb") as fp:
    pickle.dump(abc_links, fp)

# Next Steps



After realizing the meniality of scraping Wikipedia for my entire dataset, I looked to Google to see what my other options are. As stated above, I have come to learn that IMDB will not be of any use to me for this effort. A few other movie and tv databases did show up - The Movie Database (TMDB), The Open Movie Database (OMDB) and The TVDB (TVDB). Each of these APIs (seemingly) have different features/metadata available, so I erred on having more data than not and will be trying to utilize all three APIs.